# Introduction to Time Series Analysis

**Time series analysis** is a statistical technique and a branch of data analysis that focuses on studying and modeling data points collected, recorded, or observed over a sequential and equally spaced time interval. Time series data represents **observations of a variable or phenomenon at different points in time**, and the goal of time series analysis is to extract meaningful insights, patterns, and predictions from this data.

# Importing Libraries

The code begins by importing necessary Python libraries for data analysis, visualization, time series modeling, and performance evaluation. The libraries included are:


1.   **Pandas** - Pandas is a powerful data manipulation and analysis library for Python. It provides data structures and functions for working with structured data, such as tables or spreadsheets.
2.   **Numpy** - NumPy (Numerical Python) is a fundamental library for numerical and mathematical operations in Python. It provides support for arrays and matrices, essential for scientific and numerical computations.
3.   **Matplotlib** - Matplotlib is a data visualization library for creating static, animated, or interactive plots and charts. It is widely used for creating graphical representations of data.
4.   **Warnings** - The warnings module is a built-in Python library for controlling the issuance of warning messages in your code. It allows you to suppress or handle warnings, making your code cleaner and more robust.
5.   **Statsmodels** - Statsmodels is a Python library for estimating and interpreting various statistical models. It is particularly useful for time series analysis, regression analysis, and hypothesis testing.
6.   **Scikit-learn** - Scikit-Learn is a machine learning library that provides a wide range of machine learning algorithms and tools for data mining and data analysis. It is designed for easy integration into data analysis workflows.


In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import TimeSeriesSplit
from datetime import datetime, timedelta
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [62]:
# Suppress warnings
warnings.filterwarnings("ignore")

# Data Loading and Preprocessing

The dataset is loaded from a CSV file ("train.csv") into a Pandas DataFrame. The initial exploration includes displaying the first few rows and summary statistics of the dataset.

The code converts the "Order Date" column to datetime format, sorts the DataFrame by date, and then aggregates sales data into a time series using the groupby function.

In [ ]:
df = pd.read_csv("train.csv")

print(df.head())
print(df.describe())

In [64]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

df = df.sort_values('Order Date')
sales_time_series = df.groupby('Order Date')['Sales'].sum()

# Exploratory Data Analysis (EDA)

**Exploratory Data Analysis (EDA)** is a systematic and critical process of analyzing, summarizing, and visualizing data to understand its key characteristics, uncover patterns, identify anomalies, and generate hypotheses. EDA is typically the initial phase of data analysis, and it plays a fundamental role in the data science and decision-making process. Advanced techniques that involved in EDA phase are:

1.   **Rolling Statistics for Seasonality and Trends** - The **rolling mean** (also known as a moving average) is calculated to smooth out short-term fluctuations or noise in time series data. The **rolling standard deviation** measures the volatility or variability of data points within a moving window. It is used to identify periods of high or low volatility.

2.   **Autocorrelation (ACF)** - Autocorrelation measures the correlation between a time series and its lagged values. It reveals how current data points depend on previous ones.
3.   **Partial Autocorrelation (PACF)** - Partial autocorrelation, in contrast to ACF, measures the direct correlation between a time series and its lagged values while accounting for the influence of intermediate lags.



In [ ]:
# EDA
plt.figure(figsize=(12,6))
plt.plot(sales_time_series)
plt.title('Sales Over Time')
plt.xlabel('Order Date')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Rolling Statistics for Seasonality and Trends
rolling_mean = sales_time_series.rolling(window=7).mean()  # 7-day rolling mean for weekly seasonality
rolling_std = sales_time_series.rolling(window=7).std()    # 7-day rolling standard deviation
plt.figure(figsize=(12, 6))
plt.plot(sales_time_series, label='Original')
plt.plot(rolling_mean, label='7-Day Rolling Mean')
plt.plot(rolling_std, label='7-Day Rolling Std')
plt.title('Rolling Mean & Standard Deviation')
plt.xlabel('Order Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)

In [ ]:
# ACF and PACF Plots for Autocorrelation
plt.figure(figsize=(12, 6))
plt.subplot(211)
plot_acf(sales_time_series, lags=30, ax=plt.gca())
plt.subplot(212)
plot_pacf(sales_time_series, lags=30, ax=plt.gca())
plt.tight_layout()

In [68]:
# Define a date range for the next 7 days
last_date = sales_time_series.index.max()
forecast_start_date = last_date + timedelta(days=1)
forecast_end_date = last_date + timedelta(days=7)
date_range = pd.date_range(forecast_start_date, forecast_end_date)

# Holts-Winters Exponential Smoothing Forecasting

**Holt-Winters Exponential Smoothing Forecasting** is a sophisticated time series forecasting technique that incorporates three primary components: level (or base), trend, and seasonality, to model and predict future values in time series data. It is a powerful and versatile method commonly used in business and economics for short to medium-term forecasting.

In [69]:
# Fit the Holt-Winters Exponential Smoothing model
model = ExponentialSmoothing(sales_time_series, trend='add', seasonal='add', seasonal_periods=7)
model_fit = model.fit(optimized=True)

In [ ]:
# Forecast sales for the next 7 days
forecasted_sales = model_fit.forecast(steps=7)
forecast_df = pd.DataFrame({'Date': date_range, 'Forecasted Sales': forecasted_sales})
print(forecast_df)

# Cross-Validation Using TimeSeriesSplit

**Cross-Validation with TimeSeriesSplit** is employed to assess the model's performance. The time series is divided into training and validation subsets to simulate forecasting and validation. For each fold, the Holt-Winters model is fit to the training data, and forecasts are generated for the validation set. Mean Absolute Error (MAE) and Mean Squared Error (MSE) are calculated for each fold.

In [71]:
tscv = TimeSeriesSplit(n_splits=5)
mae_scores = []
mse_scores = []

for train_index, test_index in tscv.split(sales_time_series):
    train_data, validation_data = sales_time_series.iloc[train_index], sales_time_series.iloc[test_index]

    # Fit the Holt-Winters Exponential Smoothing model on the training data
    model = ExponentialSmoothing(train_data, trend='add', seasonal='add', seasonal_periods=7)
    model_fit = model.fit(optimized=True)

    # Forecast sales for the validation set (simulating the next 7 days)
    forecasted_sales = model_fit.forecast(steps=len(validation_data))

    # Calculate performance metrics
    mae = mean_absolute_error(validation_data, forecasted_sales)
    mse = mean_squared_error(validation_data, forecasted_sales)

    mae_scores.append(mae)
    mse_scores.append(mse)

# Calculate the mean and standard deviation of MAE and MSE scores
mean_mae = np.mean(mae_scores)
mean_mse = np.mean(mse_scores)
std_mae = np.std(mae_scores)
std_mse = np.std(mse_scores)

print(f"Mean Absolute Error (MAE): {mean_mae:.2f} ± {std_mae:.2f}")
print(f"Mean Squared Error (MSE): {mean_mse:.2f} ± {std_mse:.2f}")

Mean Absolute Error (MAE): 1711.65 ± 87.53
Mean Squared Error (MSE): 6385888.14 ± 1317647.57
